## Proof of concept

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [2]:
# https://deeplearningsandbox.com/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2

In [3]:
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [4]:
# organize imports
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [5]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import h5py
import os
import json
import pickle as cPickle
import datetime
import time

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
img_width, img_height = 128, 128
train_data_dir = "data/MSTAR_Chips/TRAIN_images"
validation_data_dir = "data/MSTAR_Chips/TEST_images"
nb_train_samples = 1622
nb_validation_samples = 1365 
batch_size = 32
epochs = 50

In [8]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

val_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = val_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical",
shuffle=False)

test_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode=None,
shuffle=False)

Found 1622 images belonging to 3 classes.
Found 1365 images belonging to 3 classes.
Found 1365 images belonging to 3 classes.


In [72]:
train_data_dir="data/MSTAR_Chips/TEST_images"
train_labels = os.listdir(train_data_dir)
# variables to hold features and labels
features = []
labels   = []
image_size=(128,128)

data/MSTAR_Chips/TEST_images/bmp2_tank
data/MSTAR_Chips/TEST_images/btr70_transport
data/MSTAR_Chips/TEST_images/t72_tank


## VGG16

In [8]:
model = applications.VGG16(include_top=False, input_shape = (img_width, img_height, 3))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [10]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)
# Load fine tuned pre-train weights
model_final.load_weights("vgg16_1.h5")
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [11]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [ ]:
def loadDatabase():

    numSamplesTrain = float(numImgClass*(float(train)/100))
    numSamplesTrain = round(numSamplesTrain)

    dataTest = []
    labelTest = []
    filesTest = []
    filesCount = 0
    patchesCount = 0

    for c in range(1,classes+1):
        filesTest.append([])

        for s in range(1,numImgClass+1):

            if(s < numSamplesTrain+1):
                folderTrainTest = 'Treino/'
            else:
                folderTrainTest = 'Teste/'

            for b in range(1,numblock+1):
                nameImg = preName + str(c).zfill(5) + sep + str(s) + sep + str(b)
                folderClass = preName + str(c).zfill(5) + '/'
                fullPathImg = pathImages + folderClass + folderTrainTest + nameImg + imagesFormat
                image = plt.imread(fullPathImg)

                image = image[np.newaxis]

                if(folderTrainTest == 'Treino/'):
                    dataTrain.append(image)
                    labelTrain.append(c-1)
                else:
                    dataTest.append(image)
                    labelTest.append(c-1)
                    filesTest[filesCount].append(patchesCount)
                    patchesCount += 1
        filesCount+=1

    #train
    dataTrain = np.array(dataTrain)
    labelTrain = np.array(labelTrain)

    LUT = np.arange(len(dataTrain), dtype=int)
    random.shuffle(LUT)
    randomDataTrain = dataTrain[LUT]
    randomLabelTrain = labelTrain[LUT]

    X_Train = randomDataTrain.astype("float32")/255
    Y_Train = np_utils.to_categorical(randomLabelTrain, classes)

    #test       
    dataTest = np.array(dataTest)
    labelTest = np.array(labelTest)
    filesTest = np.array(filesTest)

    X_Test = dataTest.astype("float32")/255
    Y_Test = np_utils.to_categorical(labelTest, classes)

    print("Number samples train: ",numSamplesTrain*numblock*classes)
    print("Number samples test: ",(numImgClass-numSamplesTrain)*numblock*classes)       

    return X_Train, Y_Train, X_Test, Y_Test, filesTest

In [28]:
# Save the model according to the conditions  
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
                            train_generator,
                            samples_per_epoch = nb_train_samples,
                            epochs = 1,
                            validation_data = validation_generator,
                            nb_val_samples = nb_validation_samples,
                            callbacks = [early])

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_steps=1365, callbacks=[<keras.ca..., steps_per_epoch=50, epochs=1, validation_data=<keras.pre...)`


Epoch 1/1
50/50 [==============================] - 123s - loss: 0.2782 - acc: 0.8937 - val_loss: 0.2578 - val_acc: 0.8976


In [37]:
print(len(test_generator.classes))

1365


In [21]:
pred = model_final.predict_generator(test_generator,1365)

In [ ]:
print(classification_report(pred,test_generator.classes)
print("Accuracy :",accuracy_score(pred,le.inverse_transform(test_labels)))
confusion_matrix(pred,le.inverse_transform(test_labels))

## VGG with imagenet without fine tune

In [38]:
model_ = applications.VGG16(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))

In [39]:
model_.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [40]:
feat_ext1=Model(input=model_.input, output=model_.get_layer('block5_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  if __name__ == '__main__':


In [41]:
feat_ext2=Model(input=model_.input, output=model_.get_layer('block4_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  if __name__ == '__main__':


In [42]:
feat_ext3=Model(input=model_.input, output=model_.get_layer('block3_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  if __name__ == '__main__':


In [43]:
feat_ext4=Model(input=model_.input, output=model_.get_layer('block2_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  if __name__ == '__main__':


## With Feature extraction from layer block5_pool

In [44]:
# path to training dataset
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
features = []
test_features = []
labels   = []
test_labels =[]
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext1.predict(x)
		flat = feature.flatten()
		features.append(flat)
		labels.append(label)

# loop over all the labels in the folder
for label in train_labels:
	cur_path = validation_data_dir + "/" + label
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext1.predict(x)
		flat = feature.flatten()
		test_features.append(flat)
		test_labels.append(label)
		

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank


In [45]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
le_labels = le.fit_transform(labels)
test_labels=le.transform(test_labels)

In [46]:
seed=2017

In [47]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(features, le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [48]:
# evaluate the model of test data
preds = model.predict(test_features)

In [49]:
print(classification_report(le.inverse_transform(preds),le.inverse_transform(test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(test_labels)))

                 precision    recall  f1-score   support

      bmp2_tank       0.87      0.92      0.89       550
btr70_transport       0.82      0.82      0.82       196
       t72_tank       0.97      0.91      0.94       619

    avg / total       0.91      0.90      0.90      1365

Accuracy : 0.903296703297


In [46]:
confusion_matrix(le.inverse_transform(preds),le.inverse_transform(test_labels))

array([[508,  25,  17],
       [ 34, 161,   1],
       [ 45,  10, 564]])

## Feature Extraction from layer block4_pool

In [50]:
# path to training dataset
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
features = []
test_features = []
labels   = []
test_labels =[]
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext2.predict(x)
		flat = feature.flatten()
		features.append(flat)
		labels.append(label)

# loop over all the labels in the folder
for label in train_labels:
	cur_path = validation_data_dir + "/" + label
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext2.predict(x)
		flat = feature.flatten()
		test_features.append(flat)
		test_labels.append(label)

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank


In [51]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
le_labels = le.fit_transform(labels)
test_labels=le.transform(test_labels)

In [52]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(features, le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [53]:
# evaluate the model of test data
preds = model.predict(test_features)
print(classification_report(le.inverse_transform(preds),le.inverse_transform(test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(test_labels)))
confusion_matrix(le.inverse_transform(preds),le.inverse_transform(test_labels))

                 precision    recall  f1-score   support

      bmp2_tank       0.92      0.99      0.95       547
btr70_transport       0.97      0.94      0.95       203
       t72_tank       1.00      0.94      0.97       615

    avg / total       0.96      0.96      0.96      1365

Accuracy : 0.958974358974


array([[539,   6,   2],
       [ 13, 190,   0],
       [ 35,   0, 580]])

## Feature Extraction from layer block3_pool

In [54]:
# path to training dataset
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
features = []
test_features = []
labels   = []
test_labels =[]
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext3.predict(x)
		flat = feature.flatten()
		features.append(flat)
		labels.append(label)

# loop over all the labels in the folder
for label in train_labels:
	cur_path = validation_data_dir + "/" + label
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext3.predict(x)
		flat = feature.flatten()
		test_features.append(flat)
		test_labels.append(label)

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank


In [55]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
le_labels = le.fit_transform(labels)
test_labels=le.transform(test_labels)

In [56]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(features, le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [57]:
# evaluate the model of test data
preds = model.predict(test_features)
print(classification_report(le.inverse_transform(preds),le.inverse_transform(test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(test_labels)))
confusion_matrix(le.inverse_transform(preds),le.inverse_transform(test_labels))

                 precision    recall  f1-score   support

      bmp2_tank       0.95      1.00      0.98       561
btr70_transport       0.99      0.99      0.99       196
       t72_tank       1.00      0.96      0.98       608

    avg / total       0.98      0.98      0.98      1365

Accuracy : 0.979487179487


array([[560,   1,   0],
       [  1, 195,   0],
       [ 26,   0, 582]])

In [60]:
del model,preds,test_features,test_labels  

## Feature Extraction from layer block2_pool

In [61]:
# path to training dataset
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
features = []
test_features = []
labels   = []
test_labels =[]
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext4.predict(x)
		flat = feature.flatten()
		features.append(flat)
		labels.append(label)

# loop over all the labels in the folder
for label in train_labels:
	cur_path = validation_data_dir + "/" + label
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext4.predict(x)
		flat = feature.flatten()
		test_features.append(flat)
		test_labels.append(label)

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank


In [62]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
le_labels = le.fit_transform(labels)
test_labels=le.transform(test_labels)

In [63]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(features, le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [64]:
# evaluate the model of test data
preds = model.predict(test_features)
print(classification_report(le.inverse_transform(preds),le.inverse_transform(test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(test_labels)))
confusion_matrix(le.inverse_transform(preds),le.inverse_transform(test_labels))

                 precision    recall  f1-score   support

      bmp2_tank       0.94      0.99      0.97       554
btr70_transport       0.99      0.98      0.98       198
       t72_tank       1.00      0.95      0.97       613

    avg / total       0.97      0.97      0.97      1365

Accuracy : 0.971428571429


array([[551,   2,   1],
       [  4, 194,   0],
       [ 32,   0, 581]])

## With Animal Dataset

In [4]:
model_animal = applications.VGG16(weights='imagenet',include_top=False, input_shape = (128, 128, 3))

In [5]:
model_animal.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [6]:
feat_ext1=Model(input=model_animal.input, output=model_animal.get_layer('block5_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [7]:
feat_ext2=Model(input=model_animal.input, output=model_animal.get_layer('block4_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [8]:
feat_ext3=Model(input=model_animal.input, output=model_animal.get_layer('block3_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [9]:
feat_ext4=Model(input=model_animal.input, output=model_animal.get_layer('block2_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  if __name__ == '__main__':


## Feature Extraction from layer block5_pool

In [15]:
seed=2017

In [66]:
# path to training dataset
train_data_dir='E:/Projects/Sentrana/data/animal_database/train'
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
features = []
labels   = []
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	i=0
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext1.predict(x)
		flat = feature.flatten()
		features.append(flat)
		labels.append(label)
		i=i+1
	print("{}:{}".format(label,i))

bear:105
cougar:100
cow:97
coyote:100
deer:100
elephant:100
giraffe:84
goat:99
gorilla:83
horse:100
kangaroo:90
leopard:100
lion:98
panda:97
penquin:80
sheep:68
skunk:62
tiger:100
zebra:77


In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=seed)

In [17]:
# encode the labels using LabelEncoder
targetNames = np.unique(y_train)
le = LabelEncoder()
le_labels = le.fit_transform(y_train)
test_labels=le.transform(y_test)

In [18]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(X_train, y_train)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
# evaluate the model of test data
preds = model.predict(X_test)
print(classification_report(preds,y_test))
print("Accuracy :",model.score(X_test,y_test))
confusion_matrix(preds,y_test)

             precision    recall  f1-score   support

       bear       0.96      0.79      0.87        34
     cougar       0.52      0.46      0.48        35
        cow       0.66      0.81      0.72        26
     coyote       0.82      0.75      0.78        36
       deer       0.79      0.74      0.77        42
   elephant       0.94      0.81      0.87        36
    giraffe       0.88      0.91      0.89        32
       goat       0.81      0.61      0.69        41
    gorilla       0.81      0.93      0.86        27
      horse       0.97      1.00      0.98        30
   kangaroo       0.78      0.84      0.81        25
    leopard       0.89      0.80      0.84        30
       lion       0.78      0.86      0.82        37
      panda       0.90      0.96      0.93        27
    penquin       0.89      0.89      0.89        28
      sheep       0.54      0.72      0.62        18
      skunk       0.78      0.93      0.85        15
      tiger       0.84      0.97      0.90   

array([[27,  1,  1,  1,  0,  0,  0,  0,  2,  0,  1,  0,  0,  0,  0,  0,  1,
         0,  0],
       [ 0, 16,  3,  0,  4,  0,  0,  1,  0,  0,  1,  2,  5,  0,  2,  0,  0,
         1,  0],
       [ 0,  0, 21,  0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  2,  0,
         1,  0],
       [ 0,  2,  0, 27,  2,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         2,  0],
       [ 0,  3,  1,  3, 31,  1,  1,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,
         0,  0],
       [ 1,  0,  2,  0,  1, 29,  1,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,
         0,  0],
       [ 0,  0,  0,  1,  0,  0, 29,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,
         0,  0],
       [ 0,  3,  2,  1,  0,  0,  0, 25,  1,  0,  0,  0,  2,  0,  1,  6,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 25,  0,  0,  0,  0,  0,  0,  0,  2,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 30,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  1,  0,  0,  0,  0,  1,  0,  0,  0, 21,  0,  1,  0,  0,  1

In [20]:
# Getting better accuracy with 224 * 224 around 88 %

## Feature Extraction from layer block4_pool

In [23]:
# path to training dataset
train_data_dir='E:/Projects/Sentrana/data/animal_database/train'
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
features = []
labels   = []
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext2.predict(x)
		flat = feature.flatten()
		features.append(flat)
		labels.append(label)

E:/Projects/Sentrana/data/animal_database/train/bear
E:/Projects/Sentrana/data/animal_database/train/cougar
E:/Projects/Sentrana/data/animal_database/train/cow
E:/Projects/Sentrana/data/animal_database/train/coyote
E:/Projects/Sentrana/data/animal_database/train/deer
E:/Projects/Sentrana/data/animal_database/train/elephant
E:/Projects/Sentrana/data/animal_database/train/giraffe
E:/Projects/Sentrana/data/animal_database/train/goat
E:/Projects/Sentrana/data/animal_database/train/gorilla
E:/Projects/Sentrana/data/animal_database/train/horse
E:/Projects/Sentrana/data/animal_database/train/kangaroo
E:/Projects/Sentrana/data/animal_database/train/leopard
E:/Projects/Sentrana/data/animal_database/train/lion
E:/Projects/Sentrana/data/animal_database/train/panda
E:/Projects/Sentrana/data/animal_database/train/penquin
E:/Projects/Sentrana/data/animal_database/train/sheep
E:/Projects/Sentrana/data/animal_database/train/skunk
E:/Projects/Sentrana/data/animal_database/train/tiger
E:/Projects/Sentra

In [24]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=seed)

In [25]:
# encode the labels using LabelEncoder
targetNames = np.unique(y_train)
le = LabelEncoder()
le_labels = le.fit_transform(y_train)
test_labels=le.transform(y_test)

In [26]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(X_train, y_train)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
# evaluate the model of test data
preds = model.predict(X_test)
print(classification_report(preds,y_test))
print("Accuracy :",model.score(X_test,y_test))
confusion_matrix(preds,y_test)

             precision    recall  f1-score   support

       bear       0.89      0.83      0.86        30
     cougar       0.55      0.53      0.54        32
        cow       0.62      0.61      0.62        33
     coyote       0.76      0.64      0.69        39
       deer       0.72      0.82      0.77        34
   elephant       0.94      0.88      0.91        33
    giraffe       0.85      0.82      0.84        34
       goat       0.81      0.66      0.72        38
    gorilla       0.81      0.81      0.81        31
      horse       1.00      0.82      0.90        38
   kangaroo       0.59      0.62      0.60        26
    leopard       0.89      0.83      0.86        29
       lion       0.78      0.86      0.82        37
      panda       0.83      0.80      0.81        30
    penquin       0.68      0.83      0.75        23
      sheep       0.42      0.62      0.50        16
      skunk       0.67      0.92      0.77        13
      tiger       0.84      0.89      0.86   

array([[25,  0,  1,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  1,
         0,  0],
       [ 0, 17,  2,  1,  2,  0,  0,  1,  0,  0,  1,  2,  4,  1,  0,  0,  0,
         1,  0],
       [ 0,  0, 20,  0,  0,  1,  0,  1,  1,  0,  5,  0,  0,  0,  0,  5,  0,
         0,  0],
       [ 0,  5,  0, 25,  3,  0,  0,  1,  0,  0,  1,  0,  1,  0,  0,  1,  0,
         2,  0],
       [ 0,  2,  1,  2, 28,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  1,  1,  0,  0, 29,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  1,  0,  0,  1, 28,  0,  0,  0,  0,  0,  0,  1,  0,  2,  0,
         0,  1],
       [ 0,  3,  2,  1,  3,  0,  0, 25,  0,  0,  1,  0,  1,  0,  0,  1,  0,
         1,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0, 25,  0,  0,  0,  0,  0,  3,  0,  2,
         0,  0],
       [ 0,  0,  1,  1,  2,  0,  0,  0,  0, 31,  1,  0,  0,  0,  0,  2,  0,
         0,  0],
       [ 0,  2,  1,  1,  0,  0,  1,  1,  0,  0, 16,  1,  0,  0,  1,  2

In [27]:
'''     precision    recall  f1-score   support

       bear       0.89      0.78      0.83        32
     cougar       0.52      0.59      0.55        27
        cow       0.59      0.63      0.61        30
     coyote       0.73      0.69      0.71        35
       deer       0.79      0.84      0.82        37
   elephant       0.94      0.88      0.91        33
    giraffe       0.82      0.90      0.86        30
       goat       0.87      0.71      0.78        38
    gorilla       0.71      0.88      0.79        25
      horse       1.00      0.69      0.82        45
   kangaroo       0.52      0.70      0.60        20
    leopard       0.93      0.81      0.86        31
       lion       0.90      0.86      0.88        43
      panda       0.90      0.72      0.80        36
    penquin       0.64      0.69      0.67        26
      sheep       0.29      0.58      0.39        12
      skunk       0.78      0.82      0.80        17
      tiger       0.87      0.97      0.92        34
      zebra       1.00      0.96      0.98        24

avg / total       0.81      0.78      0.79       575

Accuracy : 0.779130434783
Out[93]:
array([[25,  0,  1,  1,  0,  0,  0,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,
         0,  0],
       [ 0, 16,  0,  2,  3,  0,  0,  1,  0,  0,  0,  1,  2,  1,  0,  0,  0,
         1,  0],
       [ 0,  0, 19,  0,  0,  1,  0,  1,  1,  0,  2,  0,  0,  0,  0,  6,  0,
         0,  0],
       [ 0,  5,  0, 24,  1,  0,  0,  0,  0,  0,  2,  0,  1,  0,  0,  1,  0,
         1,  0],
       [ 0,  1,  1,  3, 31,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  1,  0,  0,  0, 29,  1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  1, 27,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,
         0,  0],
       [ 0,  4,  1,  1,  1,  0,  1, 27,  0,  0,  1,  0,  0,  0,  1,  1,  0,
         0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0, 22,  0,  0,  0,  0,  0,  2,  0,  0,
         0,  0],
       [ 0,  1,  3,  1,  1,  0,  0,  1,  0, 31,  2,  0,  0,  0,  0,  5,  0,
         0,  0],
       [ 0,  0,  3,  0,  1,  0,  0,  1,  0,  0, 14,  0,  0,  0,  1,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1, 25,  0,  0,  1,  1,  0,
         2,  0],
       [ 1,  1,  0,  0,  1,  0,  1,  0,  0,  0,  1,  0, 37,  0,  1,  0,  0,
         0,  0],
       [ 0,  2,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0, 26,  3,  1,  2,
         0,  0],
       [ 0,  0,  2,  1,  0,  0,  1,  0,  2,  0,  1,  0,  0,  1, 18,  0,  0,
         0,  0],
       [ 1,  0,  2,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  7,  1,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  1, 14,
         0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        33,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1, 23]])
         
'''
# result with 224*224

'     precision    recall  f1-score   support\n\n       bear       0.89      0.78      0.83        32\n     cougar       0.52      0.59      0.55        27\n        cow       0.59      0.63      0.61        30\n     coyote       0.73      0.69      0.71        35\n       deer       0.79      0.84      0.82        37\n   elephant       0.94      0.88      0.91        33\n    giraffe       0.82      0.90      0.86        30\n       goat       0.87      0.71      0.78        38\n    gorilla       0.71      0.88      0.79        25\n      horse       1.00      0.69      0.82        45\n   kangaroo       0.52      0.70      0.60        20\n    leopard       0.93      0.81      0.86        31\n       lion       0.90      0.86      0.88        43\n      panda       0.90      0.72      0.80        36\n    penquin       0.64      0.69      0.67        26\n      sheep       0.29      0.58      0.39        12\n      skunk       0.78      0.82      0.80        17\n      tiger       0.87      0.97 

In [29]:
del model,X_train, X_test, y_train, y_test,features,labels  

## Feature Extraction from layer block3_pool

In [30]:
# path to training dataset
train_data_dir='E:/Projects/Sentrana/data/animal_database/train'
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
features = []
labels   = []
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext3.predict(x)
		flat = feature.flatten()
		features.append(flat)
		labels.append(label)

E:/Projects/Sentrana/data/animal_database/train/bear
E:/Projects/Sentrana/data/animal_database/train/cougar
E:/Projects/Sentrana/data/animal_database/train/cow
E:/Projects/Sentrana/data/animal_database/train/coyote
E:/Projects/Sentrana/data/animal_database/train/deer
E:/Projects/Sentrana/data/animal_database/train/elephant
E:/Projects/Sentrana/data/animal_database/train/giraffe
E:/Projects/Sentrana/data/animal_database/train/goat
E:/Projects/Sentrana/data/animal_database/train/gorilla
E:/Projects/Sentrana/data/animal_database/train/horse
E:/Projects/Sentrana/data/animal_database/train/kangaroo
E:/Projects/Sentrana/data/animal_database/train/leopard
E:/Projects/Sentrana/data/animal_database/train/lion
E:/Projects/Sentrana/data/animal_database/train/panda
E:/Projects/Sentrana/data/animal_database/train/penquin
E:/Projects/Sentrana/data/animal_database/train/sheep
E:/Projects/Sentrana/data/animal_database/train/skunk
E:/Projects/Sentrana/data/animal_database/train/tiger
E:/Projects/Sentra

In [31]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=seed)

In [32]:
# encode the labels using LabelEncoder
targetNames = np.unique(y_train)
le = LabelEncoder()
le_labels = le.fit_transform(y_train)
test_labels=le.transform(y_test)

In [33]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(X_train, y_train)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
# evaluate the model of test data
preds = model.predict(X_test)
print(classification_report(preds,y_test))
print("Accuracy :",model.score(X_test,y_test))
confusion_matrix(preds,y_test)

             precision    recall  f1-score   support

       bear       0.82      0.64      0.72        36
     cougar       0.35      0.42      0.39        26
        cow       0.50      0.55      0.52        29
     coyote       0.61      0.61      0.61        33
       deer       0.56      0.71      0.63        31
   elephant       0.90      0.76      0.82        37
    giraffe       0.67      0.85      0.75        26
       goat       0.87      0.63      0.73        43
    gorilla       0.61      0.70      0.66        27
      horse       1.00      0.70      0.83        44
   kangaroo       0.37      0.36      0.36        28
    leopard       0.81      0.65      0.72        34
       lion       0.68      0.85      0.76        33
      panda       0.72      0.70      0.71        30
    penquin       0.54      0.62      0.58        24
      sheep       0.17      0.29      0.21        14
      skunk       0.61      0.85      0.71        13
      tiger       0.76      0.71      0.73   

array([[23,  0,  0,  1,  1,  0,  1,  1,  1,  0,  1,  0,  1,  2,  0,  2,  2,
         0,  0],
       [ 1, 11,  1,  2,  3,  0,  0,  1,  0,  0,  0,  1,  5,  1,  0,  0,  0,
         0,  0],
       [ 0,  0, 16,  1,  1,  1,  0,  1,  0,  0,  2,  0,  0,  2,  0,  5,  0,
         0,  0],
       [ 0,  6,  0, 20,  3,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,
         2,  0],
       [ 0,  1,  0,  4, 22,  0,  1,  0,  0,  0,  1,  0,  0,  1,  1,  0,  0,
         0,  0],
       [ 1,  2,  1,  0,  0, 28,  2,  1,  1,  0,  0,  0,  1,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  1,  0,  0,  1, 22,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  1],
       [ 0,  5,  2,  0,  2,  0,  0, 27,  0,  0,  1,  0,  0,  0,  2,  3,  0,
         1,  0],
       [ 1,  0,  0,  0,  0,  0,  1,  0, 19,  0,  2,  0,  0,  0,  4,  0,  0,
         0,  0],
       [ 0,  1,  1,  2,  2,  0,  0,  0,  0, 31,  2,  0,  0,  0,  0,  4,  0,
         1,  0],
       [ 0,  2,  4,  1,  0,  1,  2,  0,  2,  0, 10,  0,  2,  0,  1,  3

In [37]:
### With 224 * 224
'''    precision    recall  f1-score   support

       bear       0.93      0.74      0.83        35
     cougar       0.32      0.59      0.42        17
        cow       0.59      0.54      0.57        35
     coyote       0.67      0.58      0.62        38
       deer       0.54      0.81      0.65        26
   elephant       0.90      0.80      0.85        35
    giraffe       0.76      0.83      0.79        30
       goat       0.84      0.55      0.67        47
    gorilla       0.61      0.66      0.63        29
      horse       1.00      0.67      0.81        46
   kangaroo       0.30      0.36      0.33        22
    leopard       0.89      0.77      0.83        31
       lion       0.68      0.80      0.74        35
      panda       0.69      0.71      0.70        28
    penquin       0.64      0.55      0.59        33
      sheep       0.21      0.45      0.29        11
      skunk       0.56      0.67      0.61        15
      tiger       0.79      0.79      0.79        38
      zebra       0.96      0.92      0.94        24

avg / total       0.73      0.68      0.69       575

Accuracy : 0.681739130435
Out[99]:
array([[26,  0,  0,  1,  2,  0,  1,  0,  0,  0,  2,  0,  0,  0,  1,  0,  2,
         0,  0],
       [ 0, 10,  0,  1,  1,  0,  0,  1,  0,  0,  0,  1,  2,  1,  0,  0,  0,
         0,  0],
       [ 0,  0, 19,  1,  1,  1,  2,  0,  2,  0,  2,  0,  0,  0,  1,  6,  0,
         0,  0],
       [ 0,  5,  0, 22,  5,  0,  0,  1,  1,  0,  2,  0,  1,  0,  0,  0,  0,
         1,  0],
       [ 0,  1,  0,  1, 21,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  0,  0,
         0,  0],
       [ 0,  2,  0,  0,  0, 28,  1,  2,  0,  0,  0,  0,  2,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  0,  0,  0,  1, 25,  0,  0,  0,  1,  0,  1,  1,  0,  1,  0,
         0,  0],
       [ 0,  8,  1,  1,  3,  0,  0, 26,  0,  0,  2,  0,  2,  0,  1,  2,  0,
         1,  0],
       [ 1,  1,  0,  0,  0,  0,  0,  0, 19,  0,  2,  0,  0,  3,  3,  0,  0,
         0,  0],
       [ 0,  1,  2,  2,  1,  0,  0,  0,  0, 31,  2,  0,  0,  0,  0,  4,  0,
         3,  0],
       [ 0,  1,  3,  0,  0,  1,  2,  0,  1,  0,  8,  0,  2,  1,  0,  3,  0,
         0,  0],
       [ 0,  2,  0,  2,  0,  0,  0,  0,  0,  0,  1, 24,  0,  0,  1,  0,  0,
         1,  0],
       [ 1,  0,  0,  1,  2,  0,  1,  0,  0,  0,  2,  0, 28,  0,  0,  0,  0,
         0,  0],
       [ 0,  0,  1,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0, 20,  2,  0,  3,
         0,  0],
       [ 0,  0,  3,  0,  0,  0,  0,  1,  5,  0,  1,  0,  0,  1, 18,  2,  1,
         0,  1],
       [ 0,  0,  2,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  5,  2,
         0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  1,  0,  1, 10,
         0,  0],
       [ 0,  0,  0,  1,  3,  0,  0,  0,  0,  0,  0,  2,  2,  0,  0,  0,  0,
        30,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         2, 22]])
        '''

'    precision    recall  f1-score   support\n\n       bear       0.93      0.74      0.83        35\n     cougar       0.32      0.59      0.42        17\n        cow       0.59      0.54      0.57        35\n     coyote       0.67      0.58      0.62        38\n       deer       0.54      0.81      0.65        26\n   elephant       0.90      0.80      0.85        35\n    giraffe       0.76      0.83      0.79        30\n       goat       0.84      0.55      0.67        47\n    gorilla       0.61      0.66      0.63        29\n      horse       1.00      0.67      0.81        46\n   kangaroo       0.30      0.36      0.33        22\n    leopard       0.89      0.77      0.83        31\n       lion       0.68      0.80      0.74        35\n      panda       0.69      0.71      0.70        28\n    penquin       0.64      0.55      0.59        33\n      sheep       0.21      0.45      0.29        11\n      skunk       0.56      0.67      0.61        15\n      tiger       0.79      0.79  

In [38]:
del model,X_train, X_test, y_train, y_test,features,labels 

## Feature Extraction from layer block2_pool

In [40]:
# path to training dataset
train_data_dir='E:/Projects/Sentrana/data/animal_database/train'
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
features = []
labels   = []
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext4.predict(x)
		flat = feature.flatten()
		features.append(flat)
		labels.append(label)

E:/Projects/Sentrana/data/animal_database/train/bear
E:/Projects/Sentrana/data/animal_database/train/cougar
E:/Projects/Sentrana/data/animal_database/train/cow
E:/Projects/Sentrana/data/animal_database/train/coyote
E:/Projects/Sentrana/data/animal_database/train/deer
E:/Projects/Sentrana/data/animal_database/train/elephant
E:/Projects/Sentrana/data/animal_database/train/giraffe
E:/Projects/Sentrana/data/animal_database/train/goat
E:/Projects/Sentrana/data/animal_database/train/gorilla
E:/Projects/Sentrana/data/animal_database/train/horse
E:/Projects/Sentrana/data/animal_database/train/kangaroo
E:/Projects/Sentrana/data/animal_database/train/leopard
E:/Projects/Sentrana/data/animal_database/train/lion
E:/Projects/Sentrana/data/animal_database/train/panda
E:/Projects/Sentrana/data/animal_database/train/penquin
E:/Projects/Sentrana/data/animal_database/train/sheep
E:/Projects/Sentrana/data/animal_database/train/skunk
E:/Projects/Sentrana/data/animal_database/train/tiger
E:/Projects/Sentra

In [41]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=seed)

In [42]:
# encode the labels using LabelEncoder
targetNames = np.unique(y_train)
le = LabelEncoder()
le_labels = le.fit_transform(y_train)
test_labels=le.transform(y_test)

In [43]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(X_train, y_train)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [44]:
# evaluate the model of test data
preds = model.predict(X_test)
print(classification_report(preds,y_test))
print("Accuracy :",model.score(X_test,y_test))
confusion_matrix(preds,y_test)

             precision    recall  f1-score   support

       bear       0.79      0.63      0.70        35
     cougar       0.19      0.32      0.24        19
        cow       0.41      0.39      0.40        33
     coyote       0.48      0.41      0.44        39
       deer       0.21      0.40      0.27        20
   elephant       0.74      0.61      0.67        38
    giraffe       0.48      0.73      0.58        22
       goat       0.52      0.40      0.45        40
    gorilla       0.42      0.48      0.45        27
      horse       1.00      0.58      0.74        53
   kangaroo       0.19      0.16      0.17        32
    leopard       0.81      0.54      0.65        41
       lion       0.59      0.63      0.61        38
      panda       0.59      0.55      0.57        31
    penquin       0.32      0.36      0.34        25
      sheep       0.21      0.56      0.30         9
      skunk       0.44      0.67      0.53        12
      tiger       0.53      0.57      0.55   

array([[22,  0,  0,  1,  2,  0,  0,  0,  3,  0,  2,  1,  1,  0,  1,  0,  1,
         1,  0],
       [ 0,  6,  0,  2,  1,  0,  0,  1,  0,  0,  2,  1,  3,  1,  1,  0,  0,
         1,  0],
       [ 0,  0, 13,  0,  1,  3,  2,  2,  2,  0,  1,  0,  0,  1,  1,  7,  0,
         0,  0],
       [ 1,  5,  1, 16,  6,  0,  0,  2,  2,  0,  1,  0,  0,  0,  2,  0,  0,
         3,  0],
       [ 0,  2,  0,  1,  8,  1,  0,  2,  0,  0,  0,  0,  1,  2,  1,  0,  0,
         2,  0],
       [ 2,  2,  2,  0,  3, 23,  3,  0,  0,  0,  0,  0,  1,  1,  0,  0,  1,
         0,  0],
       [ 1,  0,  0,  0,  1,  1, 16,  0,  0,  0,  1,  1,  0,  0,  0,  1,  0,
         0,  0],
       [ 0,  8,  1,  1,  3,  0,  0, 16,  0,  0,  2,  0,  2,  1,  3,  2,  1,
         0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0, 13,  0,  4,  0,  0,  4,  4,  1,  0,
         0,  0],
       [ 1,  1,  2,  3,  3,  0,  0,  0,  0, 31,  3,  0,  0,  0,  0,  4,  1,
         4,  0],
       [ 0,  2,  4,  1,  1,  1,  2,  1,  1,  0,  5,  0,  6,  0,  2,  3

In [45]:
del model,X_train, X_test, y_train, y_test,features,labels 

## Fine Tune output with different blocks

In [9]:
model_ = applications.VGG16(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))

In [10]:
model_.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

## With block4_pool and MSTAR dataset

In [13]:
for layer in model_.layers[:15]:
    print(layer)
    layer.trainable = False

In [11]:
model_.layers[14]

In [98]:
## Read weights

In [97]:
import h5py
weights_path = 'vgg16_1.h5' # ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5)
f = h5py.File(weights_path)
list(f["model_weights"].keys())

['block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool',
 'block3_conv1',
 'block3_conv2',
 'block3_conv3',
 'block3_pool',
 'block4_conv1',
 'block4_conv2',
 'block4_conv3',
 'block4_pool',
 'block5_conv1',
 'block5_conv2',
 'block5_conv3',
 'block5_pool',
 'dense_1',
 'dense_2',
 'dense_3',
 'dropout_1',
 'flatten_1',
 'input_1']

In [15]:
#Adding custom Layers 
x = model_.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model_.input, output = predictions)
# Load fine tuned pre-train weights

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [16]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [ ]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_block_4.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
                            train_generator,
                            samples_per_epoch = nb_train_samples,
                            epochs = 50,
                            validation_data = validation_generator,
                            nb_val_samples = nb_validation_samples,
                            callbacks = [checkpoint,early])

Epoch 1/50
 1/50 [..............................] - ETA: 8s - loss: 1.0877 - acc: 0.5000

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., epochs=50, callbacks=[<keras.ca..., steps_per_epoch=50, validation_steps=1365)`


49/50 [============================>.] - ETA: 0s - loss: 1.0106 - acc: 0.4748

In [ ]:
model_final.save_weights("vgg16_block_4.h5")

In [ ]:
### Build the network 
# img_input = Input(shape=(256, 256, 3))
# x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
# x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
# x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
# x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
# x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
# x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# model = Model(input = img_input, output = x)

# model.summary()
"""
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
=================================================================
Total params: 260,160.0
Trainable params: 260,160.0
Non-trainable params: 0.0
"""

# layer_dict = dict([(layer.name, layer) for layer in model.layers])
# [layer.name for layer in model.layers]
"""
['input_1',
 'block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool']
"""


# list all the layer names which are in the model.
# layer_names = [layer.name for layer in model.layers]


"""
# Here we are extracting model_weights for each and every layer from the .h5 file
>>> f["model_weights"]["block1_conv1"].attrs["weight_names"]
array([b'block1_conv1/kernel:0', b'block1_conv1/bias:0'], 
      dtype='|S21')
# we are assiging this array to weight_names below 
>>> f["model_weights"]["block1_conv1"]["block1_conv1/kernel:0]
<HDF5 dataset "kernel:0": shape (3, 3, 3, 64), type "<f4">
# The list comprehension (weights) stores these two weights and bias of both the layers 
>>>layer_names.index("block1_conv1")
1
>>> model.layers[1].set_weights(weights)
# This will set the weights for that particular layer.
With a for loop we can set_weights for the entire network.
"""
# for i in layer_dict.keys():
#    weight_names = f["model_weights"][i].attrs["weight_names"]
#    weights = [f["model_weights"][i][j] for j in weight_names]
#    index = layer_names.index(i)
#    model.layers[index].set_weights(weights)